In [17]:
df_1=pd.read_csv("C:/Users/user/Desktop/인천광역시_도로 주변 조도_20211130/인천광역시_조도_202110.csv",encoding='cp949')
df_2=pd.read_csv("C:/Users/user/Desktop/인천광역시_도로 주변 조도_20211130/인천광역시_조도_202111.csv",encoding='cp949')

In [15]:
import requests
from pyproj import Transformer

API_KEY = "4D216E9B-5341-325B-B650-99F7B1F49D5F"
BASE_URL = "http://api.vworld.kr/req/data"
transformer = Transformer.from_crs("EPSG:4326", "EPSG:5181", always_xy=True)

def get_elevation_debug(lat, lon):
    x, y = transformer.transform(lon, lat)
    print(f"변환 좌표 EPSG:5181 -> X: {x}, Y: {y}")
    
    params = {
        'service': 'data',
        'version': '2.0',
        'request': 'GetFeature',
        'key': API_KEY,
        'data': 'dem',
        'geometry': f'POINT({x} {y})',
        'format': 'json'
    }
    print(f"요청 URL: {BASE_URL}")
    print(f"요청 파라미터: {params}")
    
    response = requests.get(BASE_URL, params=params)
    print(f"HTTP 상태 코드: {response.status_code}")
    data = response.json()
    print("응답 JSON 요약:", list(data.keys()))
    
    features = data.get('response', {}).get('result', {}).get('featureCollection', {}).get('features', [])
    
    if not features:
        print(f"[경고] 고도 데이터 없음: ({lat}, {lon})")
        return None
    
    elevation = features[0].get('properties', {}).get('elevation', None)
    print(f"고도: {elevation}")
    return elevation

# 서울 시청 좌표 테스트
get_elevation_debug(37.66, 126.98)



변환 좌표 EPSG:5181 -> X: 198235.27154152337, Y: 462262.4787630034
요청 URL: http://api.vworld.kr/req/data
요청 파라미터: {'service': 'data', 'version': '2.0', 'request': 'GetFeature', 'key': '4D216E9B-5341-325B-B650-99F7B1F49D5F', 'data': 'dem', 'geometry': 'POINT(198235.27154152337 462262.4787630034)', 'format': 'json'}
HTTP 상태 코드: 200
응답 JSON 요약: ['response']
[경고] 고도 데이터 없음: (37.66, 126.98)


In [9]:
import geopandas as gpd
import pandas as pd
import requests
import math
from pyproj import Transformer

# VWorld API 설정
API_KEY = '4D216E9B-5341-325B-B650-99F7B1F49D5F'  # 여기에 본인의 VWorld API 키 넣으세요
BASE_URL = 'http://api.vworld.kr/req/data'

# 좌표계 변환 (EPSG:5186 -> EPSG:4326)
transformer = Transformer.from_crs("EPSG:5186", "EPSG:4326", always_xy=True)

# Shapefile 로드
shapefile_path = 'C:/Users/user/Desktop/인천시공공데이터공모전/[오픈데이터]_이동약자_보행자네트워크_인도/인도.shp'
gdf = gpd.read_file(shapefile_path)
gdf=gdf.head()

def convert_coords(coords):
    return [transformer.transform(x, y) for x, y in coords]

def get_elevation(lon: float, lat: float):
    params = {
        'service': 'data',
        'version': '2.0',
        'request': 'GetFeature',
        'key': API_KEY,
        'data': 'dem',
        'geometry': f'POINT({lon} {lat})',
        'format': 'json'
    }
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        data = response.json()
        # Check if result exists
        if 'result' in data['response'] and data['response']['result']['featureCollection']['features']:
            elevation = data['response']['result']['featureCollection']['features'][0]['properties']['elevation']
            return elevation
        else:
            print(f"No elevation data for ({lon}, {lat}), setting to 0")
            return 0
    except Exception as e:
        print(f"Error fetching elevation for ({lon}, {lat}): {e}")
        return 0  # Elevation not found, set to 0

def calculate_slope(coords_4326, elevations):
    slopes = []
    for i in range(len(coords_4326) - 1):
        p1, p2 = coords_4326[i], coords_4326[i + 1]
        h1, h2 = elevations[i], elevations[i + 1]
        distance_2d = math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
        height_diff = h2 - h1
        slope_deg = math.degrees(math.atan2(height_diff, distance_2d)) if distance_2d != 0 else 0
        slopes.append(slope_deg)
    return sum(slopes) / len(slopes) if slopes else 0

# 슬로프 계산 및 추가
slopes_list = []
print("Processing polygons from shapefile...")
for index, row in gdf.iterrows():
    polygon = row['geometry']
    coords_5186 = list(polygon.exterior.coords)
    coords_4326 = convert_coords(coords_5186)

    elevations = [get_elevation(lon, lat) for lon, lat in coords_4326]
    avg_slope = calculate_slope(coords_4326, elevations)
    slopes_list.append(avg_slope)

# 결과를 데이터프레임에 추가
gdf['avg_slope'] = slopes_list

print(gdf.head())


Processing polygons from shapefile...
No elevation data for (126.68575440584874, 37.47503308187384), setting to 0
No elevation data for (126.68573572584266, 37.47502015599587), setting to 0
No elevation data for (126.6857342902828, 37.475023352937434), setting to 0
No elevation data for (126.68525680360948, 37.47537461722353), setting to 0
No elevation data for (126.68525216343093, 37.47537805579815), setting to 0
No elevation data for (126.68526878502784, 37.4753908317661), setting to 0
No elevation data for (126.685271084847, 37.47539259451643), setting to 0
No elevation data for (126.68528301585762, 37.47539704992014), setting to 0
No elevation data for (126.68529495918845, 37.47539559514803), setting to 0
No elevation data for (126.68537978426902, 37.475333192167376), setting to 0
No elevation data for (126.68575357761762, 37.47505821792513), setting to 0
No elevation data for (126.6857564906085, 37.475045611388666), setting to 0
No elevation data for (126.6857562443059, 37.4750343

In [2]:
import geopandas as gpd

# SHP 파일 불러오기
gdf = gpd.read_file('C:/Users/user/Desktop/[오픈데이터]_이동약자_보행자네트워크_인도/인도.shp')

# 데이터 구조 확인
print(gdf.head())
print(gdf.info())


   objectid  indo_id sido_nm sgg_nm emd_nm   edit_date edit_code   st_area_sh  \
0         1    544.0   인천광역시     서구   가좌3동  2021-12-03         A   231.591857   
1         2    542.0   인천광역시     서구   가좌3동  2021-12-03         A   300.798851   
2         3    525.0   인천광역시     서구   가좌3동  2021-12-03         A   254.342128   
3         4    526.0   인천광역시     서구   가좌3동  2021-12-03         A  1160.365578   
4         5    527.0   인천광역시     서구   가좌3동  2021-12-03         A   134.048600   

   st_length_                                           geometry  
0  123.258680  POLYGON ((172203.408 541779.503, 172201.751 54...  
1  330.778248  POLYGON ((172093.18 541650.614, 172092.559 541...  
2  153.750610  POLYGON ((172346.315 541381.2, 172345.23 54138...  
3  666.283231  POLYGON ((172271.684 541438.187, 172275.235 54...  
4   83.401540  POLYGON ((172000.55 541616.623, 171999.467 541...  
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 14875 entries, 0 to 14874
Data columns (total 10 colu

In [23]:
import os
import requests
from datetime import datetime

# 환경변수에서 인증키 불러오기

# API URL 설정
API_BASE_URL = "https://apihub.kma.go.kr/api/typ02/openApi/VilageFcstInfoService_2.0/getVilageFcst"

# 요청 파라미터 설정
params = {
    'pageNo': 1,
    'numOfRows': 1000,
    'dataType': 'XML',
    'base_date': '20250519',
    'base_time': '0500',
    'nx': 55,
    'ny': 125,
    'authKey': "rP3MkYGETA69zJGBhPwOnA"
}

try:
    # API 요청 보내기
    response = requests.get(API_BASE_URL, headers={'Content-Type': 'application/json'}, params=params)
    response.raise_for_status()  # HTTP 에러 발생 시 예외 처리
    print(response.text)  # 응답 출력

except requests.exceptions.RequestException as e:
    print(f"API 요청 오류: {e}")


<?xml version="1.0" encoding="UTF-8"?>
<response><header><resultCode>00</resultCode><resultMsg>NORMAL_SERVICE</resultMsg></header><body><dataType>XML</dataType><items><item><baseDate>20250519</baseDate><baseTime>0500</baseTime><category>TMP</category><fcstDate>20250519</fcstDate><fcstTime>0600</fcstTime><fcstValue>11</fcstValue><nx>55</nx><ny>125</ny></item><item><baseDate>20250519</baseDate><baseTime>0500</baseTime><category>UUU</category><fcstDate>20250519</fcstDate><fcstTime>0600</fcstTime><fcstValue>-3</fcstValue><nx>55</nx><ny>125</ny></item><item><baseDate>20250519</baseDate><baseTime>0500</baseTime><category>VVV</category><fcstDate>20250519</fcstDate><fcstTime>0600</fcstTime><fcstValue>2.5</fcstValue><nx>55</nx><ny>125</ny></item><item><baseDate>20250519</baseDate><baseTime>0500</baseTime><category>VEC</category><fcstDate>20250519</fcstDate><fcstTime>0600</fcstTime><fcstValue>129</fcstValue><nx>55</nx><ny>125</ny></item><item><baseDate>20250519</baseDate><baseTime>0500</baseTime

In [28]:
def get_base_time():
    now = datetime.now()
    # 기준 발표 시각 리스트 (시간, 분)
    base_times = [
        (2, 0), (5, 0), (8, 0), (11, 0),
        (14, 0), (17, 0), (20, 0), (23, 0)
    ]
    
    # API 제공 시각 = Base_time + 10분
    api_times = [(h, 10) for (h, m) in base_times]

    # 현재 시각이 API 제공 시간 이후인 가장 최신 Base_time을 찾기
    latest_base = None
    for i in range(len(api_times)):
        api_hour, api_min = api_times[i]
        api_datetime = now.replace(hour=api_hour, minute=api_min, second=0, microsecond=0)
        if now >= api_datetime:
            latest_base = base_times[i]
        else:
            break

    # 만약 최신 base_time이 없으면 (예: 00:00 ~ 02:09 사이)
    if latest_base is None:
        base_date = (now - timedelta(days=1)).strftime('%Y%m%d')
        base_time = '2300'
    else:
        base_date = now.strftime('%Y%m%d')
        base_time = f'{latest_base[0]:02}00'

    return base_date, base_time
    
print(get_base_time())

('20250519', '1100')


In [1]:
import geopandas as gpd
import pandas as pd
import requests
import xmltodict
from datetime import datetime, timedelta
from shapely.geometry import Point
import math

class LCCParameter:
    def __init__(self):
        self.Re = 6371.00877  # 지도반경 (km)
        self.grid = 5.0        # 격자간격 (km)
        self.slat1 = 30.0      # 표준위도 1 (degree)
        self.slat2 = 60.0      # 표준위도 2 (degree)
        self.olon = 126.0      # 기준점 경도 (degree)
        self.olat = 38.0       # 기준점 위도 (degree)
        self.xo = 210 / self.grid  # 기준점 X 좌표
        self.yo = 675 / self.grid  # 기준점 Y 좌표
        self.first = False      # 초기화 여부

        # 초기화
        self.PI = math.asin(1.0) * 2.0
        self.DEGRAD = self.PI / 180.0
        self.RADDEG = 180.0 / self.PI
        self.re = self.Re / self.grid
        self.init_params()

    def init_params(self):
        slat1 = self.slat1 * self.DEGRAD
        slat2 = self.slat2 * self.DEGRAD
        olon = self.olon * self.DEGRAD
        olat = self.olat * self.DEGRAD

        sn = math.tan(self.PI * 0.25 + slat2 * 0.5) / math.tan(self.PI * 0.25 + slat1 * 0.5)
        sn = math.log(math.cos(slat1) / math.cos(slat2)) / math.log(sn)
        sf = math.tan(self.PI * 0.25 + slat1 * 0.5)
        sf = math.pow(sf, sn) * math.cos(slat1) / sn
        ro = math.tan(self.PI * 0.25 + olat * 0.5)
        ro = self.re * sf / math.pow(ro, sn)

        self.sn = sn
        self.sf = sf
        self.ro = ro
        self.olon_rad = olon
        self.olat_rad = olat
        self.first = True

def lamcproj(lon, lat, code, map):
    if code == 0:  # 위경도 -> 격자
        ra = math.tan(map.PI * 0.25 + lat * map.DEGRAD * 0.5)
        ra = map.re * map.sf / math.pow(ra, map.sn)
        theta = lon * map.DEGRAD - map.olon_rad

        if theta > map.PI:
            theta -= 2.0 * map.PI
        if theta < -map.PI:
            theta += 2.0 * map.PI

        theta *= map.sn
        x = int(ra * math.sin(theta) + map.xo + 1.5)
        y = int(map.ro - ra * math.cos(theta) + map.yo + 1.5)

        return x, y

    elif code == 1:  # 격자 -> 위경도
        xn = lon - map.xo
        yn = map.ro - lat + map.yo
        ra = math.sqrt(xn * xn + yn * yn)

        if map.sn < 0.0:
            ra = -ra

        alat = math.pow((map.re * map.sf / ra), (1.0 / map.sn))
        alat = 2.0 * math.atan(alat) - map.PI * 0.5

        if abs(xn) <= 0.0:
            theta = 0.0
        else:
            if abs(yn) <= 0.0:
                theta = map.PI * 0.5
                if xn < 0.0:
                    theta = -theta
            else:
                theta = math.atan2(xn, yn)

        alon = theta / map.sn + map.olon_rad
        return alon * map.RADDEG, alat * map.RADDEG

# SHP 파일 불러오기
gdf = gpd.read_file('C:/Users/user/Desktop/인천시공공데이터공모전/[오픈데이터]_이동약자_보행자네트워크_인도/인도.shp')
gdf=gdf.head()

# 2. 좌표계 확인 및 지정
if gdf.crs is None:
    print("좌표계가 정의되지 않았습니다. WKID 5186으로 지정합니다.")
    gdf.set_crs(epsg=5186, inplace=True)
else:
    print(f"현재 좌표계: {gdf.crs}")
    if gdf.crs != 'EPSG:5186':
        print("좌표계를 WKID 5186으로 재지정합니다.")
        gdf.set_crs(epsg=5186, inplace=True, allow_override=True)

# 좌표계 변환 (TM -> WGS84)
gdf = gdf.to_crs(epsg=4326)

# 중심점 추출
gdf['centroid'] = gdf['geometry'].centroid
gdf[['lat', 'lon']] = gdf['centroid'].apply(lambda point: pd.Series([point.y, point.x]))

# 격자 좌표 변환 (WGS84 -> 기상청 격자)
map = LCCParameter()
gdf[['nx', 'ny']] = gdf.apply(lambda row: pd.Series(lamcproj(row['lon'], row['lat'], 0, map)), axis=1)

'''
# base_date와 base_time 동적 설정
def get_base_time():
    now = datetime.now()
    base_hour = (now.hour // 3) * 3
    if base_hour == 0:
        base_date = (now - timedelta(days=1)).strftime('%Y%m%d')
        base_time = '2300'
    else:
        base_date = now.strftime('%Y%m%d')
        base_time = f'{base_hour:02}00'
    return base_date, base_time

'''

def get_base_time():
    now = datetime.now()
    # 기준 발표 시각 리스트 (시간, 분)
    base_times = [
        (2, 0), (5, 0), (8, 0), (11, 0),
        (14, 0), (17, 0), (20, 0), (23, 0)
    ]
    
    # API 제공 시각 = Base_time + 10분
    api_times = [(h, 10) for (h, m) in base_times]

    # 현재 시각이 API 제공 시간 이후인 가장 최신 Base_time을 찾기
    latest_base = None
    for i in range(len(api_times)):
        api_hour, api_min = api_times[i]
        api_datetime = now.replace(hour=api_hour, minute=api_min, second=0, microsecond=0)
        if now >= api_datetime:
            latest_base = base_times[i]
        else:
            break

    # 만약 최신 base_time이 없으면 (예: 00:00 ~ 02:09 사이)
    if latest_base is None:
        base_date = (now - timedelta(days=1)).strftime('%Y%m%d')
        base_time = '2300'
    else:
        base_date = now.strftime('%Y%m%d')
        base_time = f'{latest_base[0]:02}00'

    return base_date, base_time
    
# 날씨 데이터 가져오기 함수
def get_weather(nx, ny):
    base_date, base_time = get_base_time()
    API_BASE_URL = "https://apihub.kma.go.kr/api/typ02/openApi/VilageFcstInfoService_2.0/getVilageFcst"
    params = {
        'pageNo': 1,
        'numOfRows': 1000,
        'dataType': 'XML',
        'base_date': base_date,
        'base_time': base_time,
        'nx': nx,
        'ny': ny,
        'authKey': "rP3MkYGETA69zJGBhPwOnA"  # 실제 인증키로 변경 필요
    }

    try:
        response = requests.get(API_BASE_URL, params=params)
        response.raise_for_status()
        data = xmltodict.parse(response.text)
        items = data['response']['body']['items']['item']
        weather = {}
        for item in items:
            category = item['category']
            value = item['fcstValue']
            weather[category] = value
        return weather
        
    except Exception as e:
        print(f"API 요청 오류 (nx: {nx}, ny: {ny}): {e}")
        return {}

# 날씨 데이터 추가
gdf['weather'] = gdf.apply(lambda row: get_weather(row['nx'], row['ny']), axis=1)

# 결과 출력 (확인용)
print(gdf[['lat', 'lon', 'nx', 'ny', 'weather']].head())


좌표계가 정의되지 않았습니다. WKID 5186으로 지정합니다.


C:\Users\user\AppData\Local\Temp\ipykernel_1756\377236249.py:107: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf['geometry'].centroid


         lat         lon  nx   ny  \
0  37.475213  126.685510  55  125   
1  37.474409  126.685145  55  125   
2  37.471643  126.687052  55  125   
3  37.472730  126.685124  55  125   
4  37.473607  126.683271  55  125   

                                             weather  
0  {'TMP': '17', 'UUU': '-1.2', 'VVV': '1.4', 'VE...  
1  {'TMP': '17', 'UUU': '-1.2', 'VVV': '1.4', 'VE...  
2  {'TMP': '17', 'UUU': '-1.2', 'VVV': '1.4', 'VE...  
3  {'TMP': '17', 'UUU': '-1.2', 'VVV': '1.4', 'VE...  
4  {'TMP': '17', 'UUU': '-1.2', 'VVV': '1.4', 'VE...  


In [1]:
import geopandas as gpd
import pandas as pd

# 파일 경로 및 타입 정의
files = [
    "C:/Users/user/Desktop/인천시공공데이터공모전/[오픈데이터]인천시_이동약자_보행자네트워크_연결시설18종_경사로/이동약자18종_경사로.shp",
    "C:/Users/user/Desktop/인천시공공데이터공모전/[오픈데이터]인천시_이동약자_보행자네트워크_연결시설18종_계단급경사/이동약자18종_계단급경사.shp",
    "C:/Users/user/Desktop/인천시공공데이터공모전/[오픈데이터]인천시_이동약자_보행자네트워크_연결시설18종_보도_턱/이동약자18종_보도_턱.shp",
    "C:/Users/user/Desktop/인천시공공데이터공모전/[오픈데이터]인천시_이동약자_보행자네트워크_연결시설18종_횡단보도_턱/이동약자18종_횡단보도_턱.shp"
]
types = ["경사로", "계단급경사", "보도턱", "횡단보도턱"]

# 데이터프레임 병합 준비
gdfs = []

for file, type_name in zip(files, types):
    gdf = gpd.read_file(file)

    if gdf.crs is None:
        print("좌표계가 정의되지 않았습니다. WKID 5186으로 지정합니다.")
        gdf.set_crs(epsg=5186, inplace=True)
    else:
        print(f"현재 좌표계: {gdf.crs}")
        if gdf.crs != 'EPSG:5186':
            print("좌표계를 WKID 5186으로 재지정합니다.")
            gdf.set_crs(epsg=5186, inplace=True, allow_override=True)
 
    # 계단급경사의 좌표 정리
    if type_name == "계단급경사":
        gdf['latitude'] = gdf['startlatit']
        gdf['longitude'] = gdf['startlongi']
    
    # 필요한 컬럼 선택 (hg 포함)
    if type_name in ["계단급경사", "보도턱"]:
        gdf = gdf[['objectid', 'ctprvnnm', 'signgunm', 'latitude', 'longitude', 'geometry', 'hg']]
    else:
        gdf = gdf[['objectid', 'ctprvnnm', 'signgunm', 'latitude', 'longitude', 'geometry']]
    
    # 시설 종류 추가
    gdf['type'] = type_name
    
    # 데이터프레임 리스트에 추가
    gdfs.append(gdf)

# 병합
merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

# 좌표계 통일
merged_gdf = merged_gdf.to_crs(epsg=4326)

# 결과 확인
print(merged_gdf.head())
print(merged_gdf.info())



좌표계가 정의되지 않았습니다. WKID 5186으로 지정합니다.
좌표계가 정의되지 않았습니다. WKID 5186으로 지정합니다.
좌표계가 정의되지 않았습니다. WKID 5186으로 지정합니다.
좌표계가 정의되지 않았습니다. WKID 5186으로 지정합니다.
   objectid ctprvnnm signgunm   latitude   longitude  \
0         1    인천광역시      남동구  37.421526  126.729530   
1         2    인천광역시      남동구  37.405426  126.723187   
2         3    인천광역시      남동구  37.405726  126.723106   
3         4    인천광역시     미추홀구  37.445083  126.689948   
4         5    인천광역시     미추홀구  37.445562  126.690162   

                     geometry type  hg  
0  POINT (126.72953 37.42153)  경사로 NaN  
1  POINT (126.72319 37.40543)  경사로 NaN  
2  POINT (126.72311 37.40573)  경사로 NaN  
3  POINT (126.68995 37.44508)  경사로 NaN  
4  POINT (126.69016 37.44556)  경사로 NaN  
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 20137 entries, 0 to 20136
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   objectid   20137 non-null  int64   
 1   ctprvnnm   20137 non-null  ob

In [2]:
merged_gdf.to_file("C:/Users/user/Desktop/인천시공공데이터공모전/위험요소결합.shp")

In [58]:
import geopandas as gpd

# 좌표계 맞추기 (둘 다 EPSG:4326 이라고 가정)
gdf = gdf.to_crs(epsg=4326)
merged_gdf = merged_gdf.to_crs(epsg=4326)

# 공간조인: 포인트(시설)에 폴리곤(인도 행정구역) 정보 붙이기
joined = gpd.sjoin(merged_gdf, gdf, how='left', predicate='within')

print(joined.head())


   objectid_left ctprvnnm signgunm   latitude   longitude  \
0              1    인천광역시      남동구  37.421526  126.729530   
1              2    인천광역시      남동구  37.405426  126.723187   
2              3    인천광역시      남동구  37.405726  126.723106   
3              4    인천광역시     미추홀구  37.445083  126.689948   
4              5    인천광역시     미추홀구  37.445562  126.690162   

                     geometry type  hg  index_right  objectid_right  ...  \
0  POINT (126.72953 37.42153)  경사로 NaN       7607.0          7605.0  ...   
1  POINT (126.72319 37.40543)  경사로 NaN          NaN             NaN  ...   
2  POINT (126.72311 37.40573)  경사로 NaN          NaN             NaN  ...   
3  POINT (126.68995 37.44508)  경사로 NaN       4422.0          4418.0  ...   
4  POINT (126.69016 37.44556)  경사로 NaN          NaN             NaN  ...   

   emd_nm   edit_date edit_code   st_area_sh   st_length_  \
0   남촌도림동  2021-12-03         A  2633.215298  1067.341551   
1     NaN         NaN       NaN          NaN          

In [56]:
result

,objectid,ctprvnnm,signgunm,latitude,longitude,geometry,type,hg,index_right,indo_id,sido_nm,sgg_nm,emd_nm
0,1,인천광역시,남동구,37.421526,126.729530,POINT (126.72953 37.42153),경사로,NaN,7607.0,3199.0,인천광역시,남동구,남촌도림동
1,2,인천광역시,남동구,37.405426,126.723187,POINT (126.72319 37.40543),경사로,NaN,NaN,NaN,NaN,NaN,NaN
2,3,인천광역시,남동구,37.405726,126.723106,POINT (126.72311 37.40573),경사로,NaN,NaN,NaN,NaN,NaN,NaN
3,4,인천광역시,미추홀구,37.445083,126.689948,POINT (126.68995 37.44508),경사로,NaN,4422.0,2513.0,인천광역시,미추홀구,관교동
4,5,인천광역시,미추홀구,37.445562,126.690162,POINT (126.69016 37.44556),경사로,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20132,1478,인천광역시,서구,37.595997,126.670717,POINT (126.67072 37.596),횡단보도턱,NaN,14012.0,4835.0,인천광역시,서구,마전동
20133,1479,인천광역시,서구,37.605745,126.659943,POINT (126.65994 37.60574),횡단보도턱,NaN,NaN,NaN,NaN,NaN,NaN
20134,1480,인천광역시,서구,37.605522,126.659856,POINT (126.65986 37.60552),횡단보도턱,NaN,NaN,NaN,NaN,NaN,NaN
20135,1481,인천광역시,남동구,37.433927,126.750461,POINT (126.75046 37.43393),횡단보도턱,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
import geopandas as gpd

# SHP 파일 불러오기
gdf_1 = gpd.read_file("C:/Users/user/Desktop/인천시공공데이터공모전/[오픈데이터]인천시_이동약자_보행자네트워크_연결시설18종_경사로/이동약자18종_경사로.shp")

# 데이터 구조 확인
print(gdf_1.head())
print(gdf_1.info())


   objectid ctprvnnm signgunm signgucode rdnmadr                 lnmadr  \
0         1    인천광역시      남동구      28200    None  인천광역시 남동구 남촌도림동 648-5   
1         2    인천광역시      남동구      28200    None   인천광역시 남동구 논현2동 685-0   
2         3    인천광역시      남동구      28200    None   인천광역시 남동구 논현2동 688-0   
3         4    인천광역시     미추홀구      28177    None   인천광역시 미추홀구 관교동 520-0   
4         5    인천광역시     미추홀구      28177    None   인천광역시 미추홀구 관교동 520-0   

    latitude   longitude institutio phonenumer  referenced instt_code  \
0  37.421526  126.729530     인천광역시청       None  2021-12-03       None   
1  37.405426  126.723187     인천광역시청       None  2021-12-03       None   
2  37.405726  126.723106     인천광역시청       None  2021-12-03       None   
3  37.445083  126.689948     인천광역시청       None  2021-12-03       None   
4  37.445562  126.690162     인천광역시청       None  2021-12-03       None   

  instt_nm handicapcv handicap_1                       geometry  
0     None          Y        경사로   POINT (17

In [41]:
import geopandas as gpd

# SHP 파일 불러오기
gdf_2 = gpd.read_file("C:/Users/user/Desktop/인천시공공데이터공모전/[오픈데이터]인천시_이동약자_보행자네트워크_연결시설18종_계단급경사/이동약자18종_계단급경사.shp")

# 데이터 구조 확인
print(gdf_2.head())
print(gdf_2.info())


   objectid ctprvnnm signgunm signgucode  \
0         1    인천광역시      남동구      28200   
1         2    인천광역시      남동구      28200   
2         3    인천광역시      남동구      28200   
3         4    인천광역시      남동구      28200   
4         5    인천광역시     미추홀구      28177   

                                  rdnmadr                lnmadr   hg  \
0  인천광역시 남동구 선수촌로 75, (구월동, 구월 유승한내들 퍼스티지   인천광역시 남동구 구월1동 1508  0.0   
1  인천광역시 남동구 선수촌로 75, (구월동, 구월 유승한내들 퍼스티지   인천광역시 남동구 구월1동 1508  0.0   
2                 인천광역시 남동구 성리로 50, (구월동)    인천광역시 남동구 구월1동 258  0.0   
3   인천광역시 남동구 선수촌공원로 85, (구월동, 구월아시아드선수촌1   인천광역시 남동구 구월1동 1486  0.0   
4         인천광역시 미추홀구 승학길 88, (주안동, 태평아파트)  인천광역시 미추홀구 주안7동 1286  0.0   

   startlatit  startlongi  endlatitud  endlongitu institutio phonenumer  \
0   37.446191  126.715227   37.446130  126.715227     인천광역시청       None   
1   37.446202  126.715227   37.446191  126.715227     인천광역시청       None   
2   37.448609  126.710208   37.448608  126.710230     인천광역시청       None   
3 

In [42]:
import geopandas as gpd

# SHP 파일 불러오기
gdf_3 = gpd.read_file("C:/Users/user/Desktop/인천시공공데이터공모전/[오픈데이터]인천시_이동약자_보행자네트워크_연결시설18종_보도_턱/이동약자18종_보도_턱.shp")

# 데이터 구조 확인
print(gdf_3.head())
print(gdf_3.info())


   objectid ctprvnnm signgunm signgucode rdnmadr               lnmadr   hg  \
0         1    인천광역시       중구      28110    None    인천광역시 중구 용유동 66-7  0.0   
1         2    인천광역시       중구      28110    None    인천광역시 중구 용유동 66-6  0.0   
2         3    인천광역시      계양구      28245    None  인천광역시 계양구 작전서운동 934  0.0   
3         4    인천광역시      계양구      28245    None  인천광역시 계양구 작전서운동 932  0.0   
4         5    인천광역시       서구      28260    None       인천광역시 서구 오류왕길동  0.0   

    latitude   longitude institutio phonenumer  referenced instt_code  \
0  37.442268  126.376404      인천광역시       None  2021-12-03       None   
1  37.443530  126.376250      인천광역시       None  2021-12-03       None   
2  37.532056  126.735374      인천광역시       None  2021-12-03       None   
3  37.531759  126.734737      인천광역시       None  2021-12-03       None   
4  37.568174  126.601612      인천광역시       None  2021-12-03       None   

  instt_nm handicapcv handicap_1                       geometry  
0     None          N     

In [43]:
import geopandas as gpd

# SHP 파일 불러오기
gdf_4 = gpd.read_file("C:/Users/user/Desktop/인천시공공데이터공모전/[오픈데이터]인천시_이동약자_보행자네트워크_연결시설18종_횡단보도_턱/이동약자18종_횡단보도_턱.shp")

# 데이터 구조 확인
print(gdf_4.head())
print(gdf_4.info())



   objectid ctprvnnm signgunm signgucode  \
0         1    인천광역시      남동구      28200   
1         2    인천광역시      남동구      28200   
2         3    인천광역시      남동구      28200   
3         4    인천광역시      옹진군      28720   
4         5    인천광역시      옹진군      28720   

                                  rdnmadr                 lnmadr   latitude  \
0        인천광역시 남동구 담방로 25-1, (만수동, 구두수선대)    인천광역시 남동구 만수6동 1066  37.443512   
1                 인천광역시 남동구 독곡로 51, (서창동)  인천광역시 남동구 장수서창동 563-1  37.434488   
2       인천광역시 남동구 장승남로 37-1, (만수동, 가로판매대)    인천광역시 남동구 만수6동 1067  37.443522   
3  인천광역시 옹진군 연평면 연평중앙로24번길 3, (연평리, 연평면사무    인천광역시 옹진군 연평면 411-3  37.664261   
4  인천광역시 옹진군 연평면 연평중앙로24번길 3, (연평리, 연평면사무    인천광역시 옹진군 연평면 411-3  37.664242   

    longitude institutio phonenumer  referenced instt_code instt_nm  \
0  126.738895     인천광역시청       None  2021-12-03       None     None   
1  126.749382     인천광역시청       None  2021-12-03       None     None   
2  126.735937     인천광역시청       None  2021-12-03